In [9]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime 

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

#Yahoo!Finance 模組
import yfinance as yf 
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [10]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [11]:
print(tf.__version__)

2.12.0


In [12]:
tf.random.set_seed(42)
np.random.seed(42)

In [13]:
# https://finance.yahoo.com/most-active
# https://finance.yahoo.com/crypto/
symbol = "SIRI"
# Read data
# start =dt.date(2018, 1, 1)
end = dt.date(2023, 7, 1)


df = yf.download(symbol,end=end)
df_Indicators= yf.download(symbol)
df.tail()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-26,3.83,3.95,3.80,3.91,3.91,10996000
2023-06-27,3.92,4.32,3.89,4.09,4.09,39288300
2023-06-28,4.14,4.29,4.11,4.27,4.27,21406200
2023-06-29,4.30,4.33,4.16,4.33,4.33,15686800
2023-06-30,4.37,4.55,4.32,4.53,4.53,17128200


In [14]:
df_Indicators.shape

(7265, 6)

In [15]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


# 指標副程式(資料,天數)
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
df_Indicators= df_Indicators.dropna()
df_Indicators.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
Date,,,,,,,,,,,,,,,
1994-10-20,3.625,3.625,3.625,3.625,3.197361,900,3.244612,3.256425,3.246758,3.304232,35.714299,0.608295,3.768174,2.973847,-0.171166
1994-10-21,3.875,3.875,3.875,3.875,3.417869,2000,3.260363,3.256425,3.289546,3.319626,50.000000,1.000000,3.737662,2.982308,-0.148606
1994-10-24,3.875,4.125,3.875,4.125,3.638376,27300,3.323365,3.287926,3.376769,3.362715,64.285701,1.000000,3.710884,2.998061,-0.111647
1994-10-25,4.000,4.000,4.000,4.000,3.528122,1200,3.370617,3.307614,3.414612,3.385033,58.620711,0.850192,3.710884,2.998061,-0.090213
1994-10-26,4.125,4.125,4.125,4.125,3.638376,100,3.433619,3.323365,3.470559,3.419163,57.142859,0.811111,3.710884,2.998061,-0.063596


In [8]:
df_Indicators.shape

(7224, 15)

In [16]:
df_Indicators.to_csv('SIRI-all.csv')

In [66]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(df_Indicators)
# df_Indicators[df_Indicators.columns] = scaler.transform(df_Indicators[df_Indicators.columns])
# df_Indicators.to_csv('ETH-all Standardization.csv')

In [18]:
n = len(df_Indicators)
train_df = df_Indicators[0:int(n*0.7)]
val_df = df_Indicators[int(n*0.7):int(n*0.9)]
test_df = df_Indicators[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((5056, 15), (1445, 15), (723, 15))

In [68]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [19]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')